# Task 1: Iris Classification

In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

iris = load_iris()
X = iris.data
y = iris.target.reshape(-1, 1)

scaler = StandardScaler()
X = scaler.fit_transform(X)

encoder = OneHotEncoder()
y = encoder.fit_transform(y).toarray()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        self.W1 = np.random.randn(input_size, hidden_size) * 0.01
        self.b1 = np.zeros(hidden_size)
        self.W2 = np.random.randn(hidden_size, output_size) * 0.01
        self.b2 = np.zeros(output_size)

    def relu(self, x):
        return np.maximum(0, x)

    def softmax(self, x):
        exps = np.exp(x - np.max(x))
        return exps / exps.sum(axis=1, keepdims=True)

    def forward(self, X):
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = self.relu(self.z1)
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        self.a2 = self.softmax(self.z2)
        return self.a2

    def compute_loss(self, y_true):
        m = y_true.shape[0]
        return -np.sum(y_true * np.log(self.a2 + 1e-15)) / m

    def backward(self, X, y_true, lr):
        m = y_true.shape[0]

        dz2 = self.a2 - y_true
        dw2 = np.dot(self.a1.T, dz2) / m
        db2 = np.sum(dz2, axis=0) / m

        dz1 = np.dot(dz2, self.W2.T) * (self.z1 > 0)
        dw1 = np.dot(X.T, dz1) / m
        db1 = np.sum(dz1, axis=0) / m

        self.W2 -= lr * dw2
        self.b2 -= lr * db2
        self.W1 -= lr * dw1
        self.b1 -= lr * db1

    def train(self, X, y, epochs=1000, lr=0.01):
        for epoch in range(epochs):
            output = self.forward(X)
            loss = self.compute_loss(y)
            self.backward(X, y, lr)
            if epoch % 100 == 0:
                print(f'Epoch {epoch}, Loss: {loss:.4f}')

    def predict(self, X):
        return np.argmax(self.forward(X), axis=1)

In [ ]:
nn = NeuralNetwork(input_size=4, hidden_size=10, output_size=3)

nn.train(X_train, y_train, epochs=1000, lr=0.01)

y_pred = nn.predict(X_test)
y_true = np.argmax(y_test, axis=1)
accuracy = np.mean(y_pred == y_true)
print(f'Test Accuracy: {accuracy:.4f}')

Epoch 0, Loss: 1.0987
Epoch 100, Loss: 1.0979
Epoch 200, Loss: 1.0959
Epoch 300, Loss: 1.0894
Epoch 400, Loss: 1.0669
Epoch 500, Loss: 0.9997
Epoch 600, Loss: 0.8700
Epoch 700, Loss: 0.7443
Epoch 800, Loss: 0.6542
Epoch 900, Loss: 0.5834
Test Accuracy: 0.8000


# Task 2: News Classification

In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelBinarizer

categories = ['talk.politics.misc', 'rec.sport.baseball', 'misc.forsale', 'sci.space']

newsgroups = fetch_20newsgroups(subset='all', categories=categories, remove=('headers', 'footers', 'quotes'))
X_text = newsgroups.data
y = newsgroups.target

vectorizer = CountVectorizer(max_features=1000)
X = vectorizer.fit_transform(X_text).toarray()

lb = LabelBinarizer()
y = lb.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
class TextClassifier(NeuralNetwork):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__(input_size, hidden_size, output_size)

input_size = X_train.shape[1]
nn_text = TextClassifier(input_size=input_size, hidden_size=64, output_size=4)

nn_text.train(X_train, y_train, epochs=500, lr=0.01)

y_pred = nn_text.predict(X_test)
y_true = np.argmax(y_test, axis=1)
accuracy = np.mean(y_pred == y_true)
print(f'Test Accuracy: {accuracy:.4f}')

Epoch 0, Loss: 1.3851
Epoch 100, Loss: 1.3517
Epoch 200, Loss: 1.3203
Epoch 300, Loss: 1.2818
Epoch 400, Loss: 1.2282
Test Accuracy: 0.6091
